In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
import random

In [3]:
words = open('names.txt').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)

32033

In [5]:
chars = sorted(list(set(''.join(words))))

In [6]:
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [127]:
#Build the dataset

def build_dataset(words, len_context):
    block_size = len_context # context length / Hyper parameter to tune
    X, Y = [], []
    random.shuffle(words)
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    
    
    n_tr = int(0.9 * len(words))
    print(len(words), n_tr)
    n_te = int(0.95 * len(words))
    X_tr = torch.tensor(X[:n_tr])
    Y_tr = torch.tensor(Y[:n_tr])
    
    X_dev = torch.tensor(X[n_tr: n_te])
    Y_dev = torch.tensor(Y[n_tr: n_te])
    
    X_test = torch.tensor(X[n_te:])
    Y_test = torch.tensor(Y[n_te:])
    return {"training": { "X_tr": X_tr, "Y_tr" : Y_tr}, 
             "dev": {"X_dev" : X_dev, "Y_dev" : Y_dev}, 
            "test": {"X_test" : X_test, "Y_test": Y_test}}


In [200]:
batch_size = 64
emb_len = 3
neurons = 100
training_steps = 1000000
len_context = 12

In [201]:

dataset = build_dataset(words, len_context)
X_tr = dataset["training"]["X_tr"]
Y_tr = dataset["training"]["Y_tr"]

X_dev = dataset["dev"]["X_dev"]
Y_dev = dataset["dev"]["Y_dev"]

X_test = dataset["test"]["X_test"]
Y_test = dataset["test"]["Y_test"]

32033 28829


In [202]:
# Training initialization with hyper parameters
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, emb_len), generator = g)
W1 = torch.randn((emb_len * len_context, neurons), generator = g) 
b1 = torch.randn(neurons, generator = g)
W2 = torch.randn((neurons, 27), generator = g)
b2 = torch.randn(27, generator = g)
parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

In [203]:
# Training
lossi = []
stepi = []

for i in range(training_steps):
    ix = torch.randint(0, X_tr.shape[0], (batch_size,))
    # Forward pass
    emb = C[X_tr[ix]]
    h = torch.tanh(emb.view(-1, emb_len * len_context) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y_tr[ix])
    for p in parameters:
        p.grad = None
    loss.backward()
    for p in parameters:
        lr = 0.1 if i < 100000 else 0.01
        p.data += -lr * p.grad

    stepi.append(i)
    lossi.append(loss.log10().item())

In [196]:
# Train
emb = C[X_tr]
h = torch.tanh(emb.view(-1, emb_len * len_context) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Y_tr)
print(loss.item())

2.2381668090820312


In [197]:
# Dev
emb = C[X_dev]
h = torch.tanh(emb.view(-1, emb_len * len_context) @ W1 + b1)
logits = h @ W2 + b2
val_loss = F.cross_entropy(logits, Y_dev)
print(val_loss.item())

2.274507761001587


In [198]:
import json
o = {"emb_len": emb_len,
      "batch_size": batch_size,
      "neurons": neurons,
     "training_steps": training_steps,
    "len_context": len_context
    }
loss_metrics_old[json.dumps(o)] = val_loss.item()


In [199]:
loss_metrics_old

{'{"emb_len": 4, "batch_size": 32, "neurons": 300, "training_steps": 300000, "len_context": 4}': 2.431281566619873,
 '{"emb_len": 10, "batch_size": 32, "neurons": 300, "training_steps": 300000, "len_context": 4}': 2.5124402046203613,
 '{"emb_len": 10, "batch_size": 32, "neurons": 300, "training_steps": 300000, "len_context": 8}': 2.602245569229126,
 '{"emb_len": 10, "batch_size": 32, "neurons": 300, "training_steps": 300000, "len_context": 2}': 2.4291749000549316,
 '{"emb_len": 10, "batch_size": 32, "neurons": 50, "training_steps": 300000, "len_context": 2}': 2.2822368144989014,
 '{"emb_len": 10, "batch_size": 32, "neurons": 25, "training_steps": 300000, "len_context": 2}': 2.312896728515625,
 '{"emb_len": 10, "batch_size": 32, "neurons": 50, "training_steps": 500000, "len_context": 2}': 2.2821733951568604,
 '{"emb_len": 10, "batch_size": 32, "neurons": 50, "training_steps": 500000, "len_context": 3}': 2.3285446166992188,
 '{"emb_len": 15, "batch_size": 32, "neurons": 50, "training_ste